<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/colab/mary_ft_and_context_playground_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Google colab notebook demonstrates the following:
* fine tuning on a small custom QA dataset
* experiment with use of dataset 'context' at prompt time

# Get access to Gemma via your Kaggle account:
* Log into your Kaggle account
* Request access to Gemma models using your Kaggle account. You can follow these instructions here: https://www.kaggle.com/code/nilaychauhan/get-started-with-gemma-using-kerasnlp
* You need to wait for confirmation. Note that this didn't take too long for me.
* Create an API key in your Kaggle account you will need later. You can follow these instructions here: https://christianjmills.com/posts/kaggle-obtain-api-key-tutorial/

# Ensure your Colab notebook can access Gemma:
* Add the Kaggle API key into your COLAB secrets. You can follow these instructions here: https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0

# Select an AI hardware accelerator
* Select hardware options near the top right of your Colab notebook
* I tested with A100 and it worked well. Note that I have a Colab Pro subscription.

# Install required packages

In [1]:
%%time
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3.3.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.0 MB/s eta 0:00:00
CPU times: user 48.5 ms, sys: 10.9 ms, total: 59.5 ms
Wall time: 8.14 s


# Import required packages

In [2]:
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
from keras import backend as K
import tensorflow
from IPython.display import Markdown, display
import textwrap
from google.colab import userdata
import json
import pandas as pd
import gc
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil
from google.colab import drive

# Configure this notebook

In [3]:
# set up Keras parameters recommended by Google
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00" # Avoid memory fragmentation on JAX backend.

# integrate Kaggle API
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME') # Link to KAGGLE API secret key
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY') # Link to KAGGLE API secret key

# Define useful functions

In [4]:
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

# Load/explore the dataset
* note I got this by running Mary's original notebook and copying the unpacked input folder

In [5]:
DATASET_PATH='/content/drive/MyDrive/Kaggle_X/Mary_ESSA/input/attempt-930/ESSA qna_csv.csv'
if not os.path.exists(DATASET_PATH):
  raise Exception("Cannot find the dataset")
df = pd.read_csv(DATASET_PATH)
pd.set_option('display.max_colwidth', None)
df.describe()

,Question,Answer
count,101,101
unique,95,99
top,What is the state's role in ESSA?,"Under ESSA, states get to decide the education plans for their schools within a framework provided by the federal government. The law also offers parents a chance to weigh in on these plans. This is important if your child gets special education services."
freq,2,2


In [6]:
df.head(5)

,Question,Answer
0,Does my state still have to test 95 percent of its students?,"ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments."
1,How do the students (up to 1 percent) who receive the alternate assessment count in terms of the state’s 95 percent requirement?,"As long as they meet the other requirements around alternate assessments (e.g. alignment with the state’s standards), states may count students who are assessed based on alternate academic achievement standards for purposes of meeting the 95 percent participation rate."
2,What are the related mandates or prohibitions related to Common Core?,"While states must maintain “challenging academic standards” (floor set as: at least three achievement levels in math, English/language arts, and science), there is a strong prohibition on the federal government using any of its authority to mandate or incentivize the use of particular standards. This prohibition not only applies to standards, but also assessments, curriculum, etc. The bill does note, however, that nothing in the law prohibits states from voluntarily entering into partnerships on standards."
3,What kind of alignment is required between elementary and secondary standards and higher education?,"ESSA requires that states demonstrate that their challenging academic standards are aligned with entrance requirements for public institutions of higher educations (IHEs) within that state. However, the legislation was also clear that this does permit the state’s IHEs to set or determine the state’s standards."
4,Are states required to submit their standards for approval by the U.S. Department of Education?,"No. There is clear language in the bill that no state shall be required to submit its standards to the federal government for review or approval. (Standards underlie the accountability system, which is part of the state Title I plan submitted to the Department.) Again, states must maintain challenging academic standards, but the law is very clear that states are not required to seek federal approval of their standards and can make changes to them without federal approval."


# Load the Gemma 2B base model

In [7]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

CPU times: user 9.41 s, sys: 9.64 s, total: 19 s
Wall time: 52.1 s


# Ask the non-fined-tuned, non-contexted model a question from the dataset

In [ ]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 3.81 s, sys: 1.66 ms, total: 3.81 s
Wall time: 3.79 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>The Every Student Succeeds Act (ESSA) is a federal law that replaced the No Child Left Behind Act (NCLB) in 2015. ESSA is a reauthorization of the Elementary and Secondary Education Act (ESEA), which was passed in 1965. ESSA is a comprehensive education law that aims to improve the quality of education for all students, regardless of their background or ability.<br><br>The law requires states to develop and implement plans to improve student achievement and to ensure that all students are prepared for success in college, career, and life. ESSA also requires states to collect and report data on student achievement and progress, and to use this data to inform decision-making about education policy and practice.<br><br>The law also includes a number of related mandates and prohibitions. For example, ESSA requires states to provide parents with information about their child’s progress in school, and to allow parents to participate in decisions about their child’s education. It also prohibits states from using federal funds to discriminate against students based on their race, color, national origin, or disability.<br><br>In addition, ESSA includes a number of prohibitions related to Common Core. Common Core is a set of standards for math and English language arts that was developed by the National Governors Association and the Council of Chief State School Officers. The standards were adopted by many states, but they were controversial because they were seen as too restrictive and difficult to implement.<br><br>As a result, many states have adopted their own standards, and some have even repealed Common Core altogether. ESSA prohibits states from adopting or implementing Common Core, and it also prohibits states from using federal funds to support Common Core.<br><br>In conclusion, ESSA is a comprehensive education law that aims to improve the quality of education for all students. It includes a number of related mandates and prohibitions, including those related to Common Core.</blockquote></font>

# Let's try a harder one

In [ ]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 9.63 s, sys: 3.47 ms, total: 9.63 s
Wall time: 9.58 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, your state still has to test 95 percent of its students.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). The purpose of ESSA is to ensure that all students have access to a high-quality education.<br><br>Question:<br>What are some of the requirements of ESSA?<br><br>Answer:<br>Some of the requirements of ESSA include:<br><br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education.<br>* Ensuring that all students have access to a high-quality education</blockquote></font>

# Analysis
* It seems to have gotten the answer right but
* Did it "hallucinate" and just get lucky?
* Also, the response format isn't well-formed

# Another hard one

In [ ]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)



CPU times: user 9.58 s, sys: 3.42 ms, total: 9.58 s
Wall time: 9.54 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, states are required to submit their standards for approval by the U.S. Department of Education.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that provides states with more flexibility in how they implement the No Child Left Behind Act (NCLB). ESSA also requires states to develop and implement plans to improve student achievement and close achievement gaps.<br><br>Question:<br>What is the difference between a state plan and a local plan?<br><br>Answer:<br>A state plan is a document that outlines how a state will implement ESSA. A local plan is a document that outlines how a school district or other local education agency will implement ESSA.<br><br>Question:<br>What are the requirements for a state plan?<br><br>Answer:<br>A state plan must include the following:<br>1. A description of how the state will implement ESSA.<br>2. A description of how the state will use data to improve student achievement and close achievement gaps.<br>3. A description of how the state will use Title I funds to support student achievement and close achievement gaps.<br>4. A description of how the state will use Title II funds to support teacher and principal effectiveness.<br>5. A description of how the state will use Title III funds to support English learners.<br>6. A description of how the state will use Title IV funds to support student success.<br>7. A description of how the state will use Title V funds to support student success.<br>8. A description of how the state will use Title VI funds to support student success.<br>9. A description of how the state will use Title VII funds to support student success.<br>10. A description of how the state will use Title VIII funds to support student success.<br>11. A description of how the state will use Title IX funds to support student success.<br>12. A description of how the state will use Title X funds to support student success.<br>13. A description of how the state will use Title XI funds to support student success.<br>14. A description of how the state will use Title XII funds to support student success.<br>15. A description of how the state will use Title XIII funds to support student success.<br>16. A description of how the state will use Title XIV funds to support student success.<br>17. A description of how the state will use Title XV funds to support student success.<br>18. A description of how the state will use Title XVI funds to support student success.<br>19. A description of how the state will use Title XVII funds to support student success.<br>20. A description of how the state will use Title XVIII funds to support student success.<br>21. A description of how the state will use Title XIX funds to support student success.<br>22. A description of how the state will use Title XX funds to support student success.<br>23. A description of how the state will use Title XXI funds to support student success.<br>24. A description of how the state will use Title XXII funds to support student success.<br>25. A description of how the state will use Title XXIII funds to support student success.<br>26. A description of how the state will use Title XXIV funds to support student success.<br>27. A description of how the state will use Title XXV funds to support student success.<br>28. A description of how the state will use Title XXVI funds to support student success.<br>29. A description of how the state will use Title XXVII funds to support student success.<br>30. A description of how the state will use Title XXVIII funds to support student success.<br>31. A description of how the state will use Title XXIX funds to support student success.<br>32. A description of how the state will use Title XXX funds to support student success.<br>33. A description of how the state will use Title XXXI funds to support student success.<br>34. A description of how the state will use Title XXXII funds to support student success.<br>35. A description of how the state will use Title XXXIII funds to support student success.<br>36. A description of how the state will use Title XXXIV funds to support student success.<br>37. A description of how the state will use Title XXXV funds to support student success.<br>38. A description of how the state will use Title XXXVI funds to support student success.<br>39. A description of how the state will use Title XXXVII funds to support student success.<br>40. A description of how the state will use Title XXXVIII funds to support student success</blockquote></font>

# Analysis
* It got the answer wrong
* Clearly it "hallucinated" and just sounded confident
* Also, the response format isn't concise and it rambles

# Prepare dataset for fine-tuning

In [8]:
df.dropna(inplace=True)
print(df.shape)

template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"
pre = '''The following is an excerpt from a conversation of a user with an AI assistant. '''\
      '''The assistant that can answer questions about ESSA. '''\
      '''ESSA stands for the Every Student Succeeds Act.'''

# format each training string, put them all into a list
ft_all_data = []
for idx, row in df.iterrows():
  ft_item = template.format(pre=pre, question=row['Question'], answer=row['Answer'])
  ft_all_data.append(ft_item)

# double-check
print("----")
print(ft_all_data[0])
print("----")
print(ft_all_data[1])
print("----")
print(ft_all_data[2])
print("----")
print(ft_all_data[-1])


(101, 2)
----
The following is an excerpt from a conversation of a user with an AI assistant. The assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.

Question:
Does my state still have to test 95 percent of its students? 

Answer:
ESSA requires that a state’s accountability system must measure the performance of 95 percent of students by looking at a variety of indicators. One of the indicators is “academic achievement as measured by proficiency on the annual assessments.” For this reason, in order to measure the overall achievement of 95 percent of students, 95 percent must take the annual assessments. 
----
The following is an excerpt from a conversation of a user with an AI assistant. The assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.

Question:
How do the students (up to 1 percent) who receive the alternate assessment count in terms of the state’s 95 percent requirement? 

Answer:
As long 

# Fine-tune the Gemma 2B base model

In [9]:
%%time

# Configure LORA rank=4 as fine-tuning method
gemma_lm.backbone.enable_lora(rank=4)

# Limit the input sequence length to X (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1024
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(ft_all_data, epochs=1, batch_size=1)

101/101 ━━━━━━━━━━━━━━━━━━━━ 64s 123ms/step - loss: 0.3031 - sparse_categorical_accuracy: 0.5013
CPU times: user 1min 38s, sys: 2.68 s, total: 1min 41s
Wall time: 1min 4s


# Query the fine-tuned model

In [10]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='What are the related mandates or prohibitions related to Common Core?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 56.2 s, sys: 1.04 s, total: 57.3 s
Wall time: 35.5 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>What are the related mandates or prohibitions related to Common Core?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>ESSA requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of ESSA?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>ESSA is a federal law that requires states to develop and implement a plan to ensure that all students are proficient in reading and math by the end of third grade.<br><br>Question:<br>What is the purpose of the Common Core?<br><br>Answer:<br>Common Core is a set of standards that states and districts use to teach reading and math.</blockquote></font>

In [11]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Does my state still have to test 95 percent of its students?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 10.7 s, sys: 2.13 ms, total: 10.7 s
Wall time: 10.7 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Does my state still have to test 95 percent of its students?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, your state still has to test 95 percent of its students.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that replaced No Child Left Behind (NCLB). ESSA is designed to help all students succeed in school.<br><br>Question:<br>What is the purpose</blockquote></font>

In [12]:
%%time
template = "{pre}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"

prompt = template.format(pre='''You are an AI assistant that can answer questions about ESSA.'''
                            ''' ESSA stands for the Every Student Succeeds Act.''',
                         question='Are states required to submit their standards for approval by the U.S. Department of Education?',
                         answer='')
completion = gemma_lm.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 10.6 s, sys: 5.16 ms, total: 10.6 s
Wall time: 10.6 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>You are an AI assistant that can answer questions about ESSA. ESSA stands for the Every Student Succeeds Act.<br><br>Question:<br>Are states required to submit their standards for approval by the U.S. Department of Education?<br><br>Answer:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>Yes, states are required to submit their standards for approval by the U.S. Department of Education.<br><br>Question:<br>What is the purpose of the Every Student Succeeds Act?<br><br>Answer:<br>The Every Student Succeeds Act (ESSA) is a federal law that provides states with more flexibility in how they implement the No Child Left Behind Act (NCLB). ESSA also requires states to develop and implement plans to improve student achievement and close achievement gaps.<br><br>Question:<br>What is the difference between a state plan and a local plan?<br><br>Answer:<br>A state plan is a document that outlines how a state will implement ESSA. A local plan is a document that outlines how a school district or other local education agency will implement ESSA.<br><br>Question:<br>What are the requirements for a state plan?<br><br>Answer:<br>A state plan must include the following:<br>1. A description of how the state will implement ESSA.<br>2. A description of how the state will use data to improve student achievement and close achievement gaps.<br>3. A description of how the state will use Title I funds to support student achievement and close achievement gaps.<br>4. A description of how the state will use Title II funds to support teacher and principal effectiveness.<br>5. A description of how the state will use Title III funds to support English learners.<br>6. A description of how the state will use Title IV funds to support student success.<br>7. A description of how the state will use Title V funds to support student success.<br>8. A description of how the state will use Title VI funds to support student success.<br>9. A description of how the state will use Title VII funds to support student success.<br>10. A description of how the state will use Title VIII funds to support student success.<br>11. A description of how the state will use Title IX funds to support student success.<br>12. A description of how the state will use Title X funds to support student success.<br>13. A description of how the state will use Title XI funds to support student success.<br>14. A description of how the state will use Title XII funds to support student success.<br>15. A description of how the state will use Title XIII funds to support student success.<br>16. A description of how the state will use Title XIV funds to support student success.<br>17. A description of how the state will use Title XV funds to support student success.<br>18. A description of how the state will use Title XVI funds to support student success.<br>19. A description of how the state will use Title XVII funds to support student success.<br>20. A description of how the state will use Title XVIII funds to support student success.<br>21. A description of how the state will use Title XIX funds to support student success.<br>22. A description of how the state will use Title XX funds to support student success.<br>23. A description of how the state will use Title XXI funds to support student success.<br>24. A description of how the state will use Title XXII funds to support student success.<br>25. A description of how the state will use Title XXIII funds to support student success.<br>26. A description of how the state will use Title XXIV funds to support student success.<br>27. A description of how the state will use Title XXV funds to support student success.<br>28. A description of how the state will use Title XXVI funds to support student success.<br>29. A description of how the state will use Title XXVII funds to support student success.<br>30. A description of how the state will use Title XXVIII funds to support student success.<br>31. A description of how the state will use Title XXIX funds to support student success.<br>32. A description of how the state will use Title XXX funds to support student success.<br>33. A description of how the state will use Title XXXI funds to support student success.<br>34. A description of how the state will use Title XXXII funds to support student success.<br>35. A description of how the state will use Title XXXIII funds to support student success.<br>36. A description of how the state will use Title XXXIV funds to support student success.<br>37. A description of how the state will use Title XXXV funds to support student success.<br>38. A description of how the state will use Title XXXVI funds to support student success.<br>39. A description of how the state will use Title XXXVII funds to support student success.<br>40. A description of how the state will use Title XXXVIII funds to support student success</blockquote></font>

# Analysis
* The results on the fine-tuned model aren't great both in terms of factual correctness and conciseness of response
* Possible issues/resolutions
  * fine-tuning dataset might be too small
  * try different fine-tuning parameters
  * try the Gemma instruction tuned version